In [150]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import country_converter as coco

## Get the data
The function bellow is used to extract the first table present in a wikipedia Page

In [151]:

def getDataFrameFromWikipedia(wikipedia_url, table_no=1):
    """
    returns a dataframe of the data from the wikipedia page. \n
    optionaly, if there are multiple tables on the page, you can specify which table to use in `table_no`.
    """
    response = requests.get(wikipedia_url)
    print(response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.findAll('table', {'class': 'wikitable'})
    
    if table_no >= 1 and table_no <= len(tables):
        df = pd.read_html(str(tables[table_no - 1]))[0]
        return df
    else:
        return None

## Normalise the country
- As there are no standerdised way to get countries, we use a library called `country_converter` to convert countries for us.
- Some country might not exist (Channel Islands, European Union, World...), in that case, we just drop the whole record for this country

In [152]:
def normaliseCountryNames(df):
    df.columns.values[0] = 'Country'
    df['Country'] = df['Country'].apply(lambda x: coco.convert(names=x, to='name_short', not_found='not found'))
    df = df[df['Country'] != 'not found']
    return df

In [153]:
def keepOnlyNumberInLastColumn(df):
    last_col = df.columns[-1]
    df.loc[:, last_col] = df[last_col].str.extract('(\d+)')
    df.loc[:, last_col] = pd.to_numeric(df[last_col], errors='coerce')

    return df

## All column

In [154]:
the_final_table = []

In [155]:
## GET GDP PER CAPITA

gdp_per_capita = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita")

df= gdp_per_capita

df = df.drop(["IMF[4][5]", "United Nations[7]"], axis = 1)
df = df.drop(0)
df.columns = df.columns.droplevel(-1)
df.columns = [*df.columns[:-1], 'years']
df.columns.values[0] = 'Country'

df = df.drop(["UN Region", "years"], axis = 1)


df = normaliseCountryNames(df)

the_final_table = df

print(df.head(10000)) 

200


/var/folders/rl/9_sc5q9x00vg1qz14xd82nhw0000gn/T/ipykernel_55982/563173382.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["IMF[4][5]", "United Nations[7]"], axis = 1)
Channel Islands not found in regex
European Union[n 1] not found in regex
World not found in regex
Zanzibar not found in regex


           Country World Bank[6]
1           Monaco        234317
2    Liechtenstein        184083
3       Luxembourg        126426
4          Bermuda        118846
5          Ireland        104039
..             ...           ...
219     Madagascar           505
220    South Sudan          1072
221   Sierra Leone           461
222    Afghanistan           364
223        Burundi           238

[219 rows x 2 columns]


In [156]:
# List of countries by Internet connection speeds, Fixed broadband, Average download speed (Mbit/s) (Ookla).

internet_speed = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_Internet_connection_speeds")

df = internet_speed
df = df.drop(["Rank", "Averagedownloadspeed(Mbit/s)(M-Lab)[2]", "Averagedownloadspeed(Mbit/s)(SpeedTestNet.io)", "Averagedownloadspeed(Mbit/s)(Speed-Test-Pros.com)[4]"], axis = 1)
df.columns = [*df.columns[:-1], "Average download speed (Mbit/s)"]
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000)) 

200
             Country  Average download speed (Mbit/s)
0            Romania                           178.90
1        South Korea                           241.58
2          Hong Kong                           265.17
3             Monaco                           220.35
4          Singapore                           259.11
..               ...                              ...
96          Dominica                            37.62
97   North Macedonia                            37.20
98            Greece                            36.73
99        Bangladesh                            36.02
100          Senegal                            35.32

[101 rows x 2 columns]


In [157]:
# List of countries by alcohol consumption per capita, Recorded per capita consumption of pure alcohol (litres) per adult 15 years of age and over per year, 2016.
alcohol_consumption = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_alcohol_consumption_per_capita")

df = alcohol_consumption

df = df.drop(["1996[7]"], axis = 1)
df.columns = [*df.columns[:-1], "Alcohiol consumption per capita (litres) 2016"]
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))

200


Netherlands Antilles not found in regex


         Country Alcohiol consumption per capita (litres) 2016
0    Afghanistan                                           0.2
1        Albania                                           7.5
2        Algeria                                           0.9
3        Andorra                                          11.3
4         Angola                                           6.4
..           ...                                           ...
186    Venezuela                                           5.6
187      Vietnam                                           8.3
188        Yemen                                           0.1
189       Zambia                                           4.8
190     Zimbabwe                                           4.8

[190 rows x 2 columns]


In [158]:
# List of countries by intentional homicide rate, Intentional homicide victims per 100,000 inhabitants. From UNODC, rate.

homicide_rate = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate",2)

df = homicide_rate

df = df.drop(["Region", "Subregion", "Year", "Count"], axis = 1)
df.columns = [*df.columns[:-1], "Intentional homicide victims per 100,000 inhabitants"]
df['Location'] = df['Location'].str.replace('*', '', regex=False).str.strip()
df = df.rename(columns={'Location': 'Country'})
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))

200


Northern Ireland not found in regex
Scotland not found in regex


            Country  Intentional homicide victims per 100,000 inhabitants
0       Afghanistan                                                4.0   
1           Albania                                                2.3   
2           Algeria                                                1.6   
3    American Samoa                                                0.0   
4           Andorra                                                2.6   
..              ...                                                ...   
201       Venezuela                                               19.3   
202         Vietnam                                                1.5   
203           Yemen                                                6.3   
204          Zambia                                                5.2   
205        Zimbabwe                                                6.1   

[204 rows x 2 columns]


In [159]:
# List of countries by military expenditures, List by the Stockholm International Peace Research Institute, % of GDP.

list_of_military_expendature = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_with_highest_military_expenditures", 1)

df = list_of_military_expendature

df = df[['Country', '% of GDP']]
df.columns = [*df.columns[:-1], "Military Expenditure % GDP"]

normaliseCountryNames(df)
the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df)

World total not found in regex


200
           Country  Military Expenditure % GDP
0        not found                         2.2
1    United States                         3.5
2            China                         1.6
3           Russia                         4.1
4            India                         2.4
5     Saudi Arabia                         7.4
6   United Kingdom                         2.2
7          Germany                         1.4
8           France                         1.9
9      South Korea                         2.7
10           Japan                         1.1
11         Ukraine                        34.0
12           Italy                         1.7
13       Australia                         1.9
14          Canada                         1.2
15          Israel                         4.5
16           Spain                         1.5
17          Brazil                         1.1
18          Poland                         2.4
19     Netherlands                         1.6
20       

/var/folders/rl/9_sc5q9x00vg1qz14xd82nhw0000gn/T/ipykernel_55982/4042162181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'] = df['Country'].apply(lambda x: coco.convert(names=x, to='name_short', not_found='not found'))


In [160]:
## GET Human Development Index,

human_dev_index = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_Human_Development_Index", 2)

df= human_dev_index
# print(df)


df = df.drop(["Rank"], axis=1)

df = df[[('Nation', 'Nation'), ('HDI', '2021 data (2022\xa0report)\u200b[2]')]]
df.columns = df.columns.droplevel(0)

df.columns = [*df.columns[:-1], "Human Development Index"]
df = normaliseCountryNames(df)


the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000)) 





200


/var/folders/rl/9_sc5q9x00vg1qz14xd82nhw0000gn/T/ipykernel_55982/1017324049.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["Rank"], axis=1)


                      Country  Human Development Index
0                 Switzerland                    0.962
1                      Norway                    0.961
2                     Iceland                    0.959
3                   Hong Kong                    0.952
4                   Australia                    0.951
..                        ...                      ...
186                   Burundi                    0.426
187  Central African Republic                    0.404
188                     Niger                    0.400
189                      Chad                    0.394
190               South Sudan                    0.385

[191 rows x 2 columns]


In [161]:
# Democracy Index, Democracy Index, 2020

democracy_index = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/The_Economist_Democracy_Index", 4)

df= democracy_index
df = df[['Country','2020']]
df.columns = [*df.columns[:-1], "Democracy Index (2020)"]
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000)) 



200
           Country  Democracy Index (2020)
0           Canada                    9.24
1    United States                    7.92
2          Austria                    8.16
3          Belgium                    7.51
4           Cyprus                    7.56
..             ...                     ...
162       Tanzania                    5.10
163           Togo                    2.80
164         Uganda                    4.94
165         Zambia                    4.86
166       Zimbabwe                    3.16

[167 rows x 2 columns]


/var/folders/rl/9_sc5q9x00vg1qz14xd82nhw0000gn/T/ipykernel_55982/4042162181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'] = df['Country'].apply(lambda x: coco.convert(names=x, to='name_short', not_found='not found'))


In [162]:
# List of countries by tertiary education attainment,Countries by level of tertiary education, at least a 2-year tertiary degree or its equivalent %.

tertiary_education_attainment = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_tertiary_education_attainment", 2)

df= tertiary_education_attainment
df = df[[('Country', 'Country'), ('Ages 25–34:\xa0% equivalent to a degree course lasting at least:', '2 years')]]
df.columns = [*df.columns[:-1], "tertiary education attainment, 2 y"]
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000)) 

200
           Country  tertiary education attainment, 2 y
0        Australia                                48.0
1          Austria                                39.0
2          Belgium                                43.0
3           Brazil                                16.0
4           Canada                                59.0
5            Chile                                36.0
6            China                                35.8
7         Colombia                                27.0
8       Costa Rica                                28.0
9          Czechia                                31.0
10         Denmark                                44.0
11         Estonia                                41.0
12         Finland                                41.0
13          France                                45.0
14         Germany                                30.0
15          Greece                                40.0
16         Hungary                                32.0
17    

/var/folders/rl/9_sc5q9x00vg1qz14xd82nhw0000gn/T/ipykernel_55982/4042162181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'] = df['Country'].apply(lambda x: coco.convert(names=x, to='name_short', not_found='not found'))


In [163]:
# Importance of religion by country, Countries/Districts, Yes important.

importance_religion = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Importance_of_religion_by_country")

df= importance_religion
df = df.drop(["No, unimportant[1]"], axis=1)
df.columns = [*df.columns[:-1], "Importance of religion"]
df = normaliseCountryNames(df)
df = keepOnlyNumberInLastColumn(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000)) 

200
        Country  Importance of religion
0        Sweden                      17
1       Denmark                      19
2       Estonia                      16
3         Japan                      24
4     Hong Kong                      24
..          ...                     ...
107   Sri Lanka                      99
108       Yemen                      99
109  Bangladesh                      99
110   Indonesia                      99
111       Niger                      99

[112 rows x 2 columns]


/var/folders/rl/9_sc5q9x00vg1qz14xd82nhw0000gn/T/ipykernel_55982/3315460159.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, last_col] = pd.to_numeric(df[last_col], errors='coerce')


In [164]:
# Christianity by country, UN members and dependent territories, % Chriastian.

christianity_by_country = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Christianity_by_country", 3)

df= christianity_by_country
df = df[['Country or entity', '% Christian']]
df.columns = [*df.columns[:-1], "Importance of religion"]
df = normaliseCountryNames(df)
df = keepOnlyNumberInLastColumn(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000)) 

200


Europe not found in regex
Latin America and the Caribbean not found in regex
Africa not found in regex
Asia not found in regex
North America not found in regex
Oceania not found in regex
Middle East-North Africa not found in regex
Total not found in regex


            Country  Importance of religion
0       Afghanistan                     0.0
1           Albania                    17.0
2           Algeria                     0.0
3    American Samoa                    98.0
4           Andorra                    89.0
..              ...                     ...
190       Venezuela                    88.0
191         Vietnam                     7.0
192           Yemen                     0.0
193          Zambia                    95.0
194        Zimbabwe                    87.0

[195 rows x 2 columns]


/var/folders/rl/9_sc5q9x00vg1qz14xd82nhw0000gn/T/ipykernel_55982/4042162181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'] = df['Country'].apply(lambda x: coco.convert(names=x, to='name_short', not_found='not found'))
/var/folders/rl/9_sc5q9x00vg1qz14xd82nhw0000gn/T/ipykernel_55982/3315460159.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, last_col] = pd.to_numeric(df[last_col], errors='coerce')


In [166]:
print(the_final_table.head(10000))

the_final_table.to_csv('the_final_table.csv', index=False)


                     Country World Bank[6]  Average download speed (Mbit/s)  \
0                     Monaco        234317                           220.35   
1              Liechtenstein        184083                           193.79   
2                 Luxembourg        126426                           154.26   
3                    Bermuda        118846                              NaN   
4                    Ireland        104039                           112.81   
..                       ...           ...                              ...   
230               St. Helena           NaN                              NaN   
231  St. Pierre and Miquelon           NaN                              NaN   
232                not found           NaN                              NaN   
233         Falkland Islands           NaN                              NaN   
234                 Pitcairn           NaN                              NaN   

    Alcohiol consumption per capita (litres) 2016  